In [1]:
# prompt: file upload

from google.colab import files
uploaded = files.upload()
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
#  print(uploaded[fn]) # To see the file contents as bytes
#  print(uploaded[fn].decode("utf-8")) # To see the file contents as string, use if the file is a text file




Saving nietzsche.txt to nietzsche.txt
User uploaded file "nietzsche.txt" with length 600901 bytes


In [3]:
import keras
from keras import layers

import numpy as np
import random
import io

"""
## Prepare the data
"""


path = keras.utils.get_file(
    "nietzsche.txt",
    origin="https://s3.amazonaws.com/text-datasets/nietzsche.txt",
)

600901/600901 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [4]:
with io.open(path, encoding="utf-8") as f:
    text = f.read().lower()
text = text.replace("\n", " ")  # We remove newlines chars for nicer display
print("Corpus length:", len(text))

chars = sorted(list(set(text)))
print("Total chars:", len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))


Corpus length: 600893
Total chars: 56


In [5]:
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i : i + maxlen])
    next_chars.append(text[i + maxlen])
print("Number of sequences:", len(sentences))

x = np.zeros((len(sentences), maxlen, len(chars)), dtype="bool")
y = np.zeros((len(sentences), len(chars)), dtype="bool")
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1



Number of sequences: 200285


In [6]:

model = keras.Sequential(
    [
        keras.Input(shape=(maxlen, len(chars))),
        layers.LSTM(128),
        layers.Dense(len(chars), activation="softmax"),
    ]
)
optimizer = keras.optimizers.RMSprop(learning_rate=0.01)
model.compile(loss="categorical_crossentropy", optimizer=optimizer)

"""
## Prepare the text sampling function
"""


'\n## Prepare the text sampling function\n'

In [7]:

def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype("float64")
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)


"""
## Train the model
"""

epochs = 40
batch_size = 128



In [ ]:

for epoch in range(epochs):
    model.fit(x, y, batch_size=batch_size, epochs=1)
    print()
    print("Generating text after epoch: %d" % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print("...Diversity:", diversity)

        generated = ""
        sentence = text[start_index : start_index + maxlen]
        print('...Generating with seed: "' + sentence + '"')

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.0
            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]
            sentence = sentence[1:] + next_char
            generated += next_char

        print("...Generated: ", generated)
        print("-")

1565/1565 ━━━━━━━━━━━━━━━━━━━━ 190s 120ms/step - loss: 2.2693

Generating text after epoch: 0
...Diversity: 0.2
...Generating with seed: "omething that will be hidden from no aug"
...Generated:  h the self-and indelf the self-and so the self-and and so the sense of the self-and so the self-and and the some and and the such as a prease of extain of the self and indelf and as a self-and and the the self-which the self-which the self-can and some the self-and and indelf is the self-and intally so the more and is the self-and and the self-conself--which as a sersions of the self-and and in th
-
...Diversity: 0.5
...Generating with seed: "omething that will be hidden from no aug"
...Generated:  hing as love the free streated one of of the who not be of self-is the the nature of the self-can and which the dood the podition of the consely and sand who which which as a panse of gree men which the side who thooly and some though the consultaring whene the great has religing the deed and mode al